In [ ]:
from os import environ

import astropy.io.fits as fits
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from astropy.cosmology import Planck15 as cosmo

from galaxy_cluster_matching import match_galaxies_and_clusters
from completeness_new import calculate_completeness_of_objects
from mass_function import get_weighted_mass_histogram, get_region_volume, get_cluster_volume
from selection_function import get_mass_luminosity_cutoff, get_distance_from_mass, filter_for_selection_function, \
    get_mass_luminosity_histogram
from constants import MASS_BINS, Z_MAX, LOG_MASS_LUMINOSITY_RATIO_BINS

# some parameters for plotting
plt.rcParams.update({
    'font.size': 18,  # Font size for text
    'axes.titlesize': 18,  # Font size for axes titles
    'axes.labelsize': 16,  # Font size for x and y labels
    'xtick.labelsize': 16,  # Font size for x tick labels
    'ytick.labelsize': 16,  # Font size for y tick labels
    'legend.fontsize': 16,  # Font size for legend text
    'figure.figsize': (15, 8),  # Default figure size
    'text.usetex': False
})
GAMA_Full_raw = fits.open('/home/farnoosh/farnoosh/Master_Thesis_all/Data/GAMA/gkvScienceCatv02/gkvScienceCatv02.fits')[
    1].data
GAMA_raw = fits.open(
    '/home/farnoosh/farnoosh/Master_Thesis_all/Data/GAMA/merged/gkvscienceStellarmass_Morphology/fullmerged_gkvscienceStellarmass_Morphologyv02_shrinked.fits')[
    1].data  #galaxy

# eFEDS = fits.open('/home/farnoosh/farnoosh/Master_Thesis_all/Data/eFEDS/Mathias_Klug/efeds_members.fit')[1].data    # check of efeds is deeper than erass1
eRASS1 = fits.open(
    '/home/farnoosh/farnoosh/Master_Thesis_all/Data/eRASS/merged_primary&optical_clusters/merged_optical_primary_clusters.fits')[
    1].data  #cluster
print(GAMA_raw['flux_rt'])
# Make dataframes

# the survey that has full observation with all different types of objects
gama_full_raw_df_unmasked = pd.DataFrame({
    'uberID': GAMA_Full_raw['uberID'].byteswap().newbyteorder(),
    'uberclass': GAMA_Full_raw['uberclass'].byteswap().newbyteorder(),
    'RA': GAMA_Full_raw['RAcen'].byteswap().newbyteorder(),
    'DEC': GAMA_Full_raw['Deccen'].byteswap().newbyteorder(),
    'z': GAMA_Full_raw['Z'].byteswap().newbyteorder(),
    'duplicate': GAMA_Full_raw['duplicate'].byteswap().newbyteorder(),
    'mask': GAMA_Full_raw['mask'].byteswap().newbyteorder(),
    'starmask': GAMA_Full_raw['starmask'].byteswap().newbyteorder(),
    'flux': GAMA_Full_raw['flux_rt'].byteswap().newbyteorder(),  # Jansky
})
# remove all the objects that do not have flux
gama_full_raw_df_unmasked = gama_full_raw_df_unmasked[gama_full_raw_df_unmasked['flux'].notna()]

# the small survey that has all the parameters that we need
gama_df_unmasked = pd.DataFrame({
    'uberID': GAMA_raw['uberID'].byteswap().newbyteorder(),
    'RA': GAMA_raw['RAcen'].byteswap().newbyteorder(),
    'DEC': GAMA_raw['Deccen'].byteswap().newbyteorder(),
    'z': GAMA_raw['Z'].byteswap().newbyteorder(),
    'mstar': GAMA_raw['mstar'].byteswap().newbyteorder(),
    'flux': GAMA_raw['flux_rt'].byteswap().newbyteorder(),  # Jansky
    'log_age': GAMA_raw['logage'].byteswap().newbyteorder(),
    'log_met': GAMA_raw['logtau'].byteswap().newbyteorder(),
    'log_met': GAMA_raw['logmet'].byteswap().newbyteorder(),
    'morphology': GAMA_raw['FINAL_CLASS'].byteswap().newbyteorder(),
    'uberclass': GAMA_raw['uberclass'].byteswap().newbyteorder(),  # Add missing columns
    'duplicate': GAMA_raw['duplicate'].byteswap().newbyteorder(),
    'mask': GAMA_raw['mask'].byteswap().newbyteorder(),
    'starmask': GAMA_raw['starmask'].byteswap().newbyteorder(),
    'NQ': GAMA_raw['NQ'].byteswap().newbyteorder(),
})

# Remove rows where flux is NaN
gama_df_unmasked = gama_df_unmasked[gama_df_unmasked['flux'].notna()]  # Jansky
gama_df_unmasked['comoving_distance'] = cosmo.comoving_distance(gama_df_unmasked['z']).value  # Mpc

# load the clusters of galaxies data from the fits object into dataframes
cluster_df_raw = pd.DataFrame({
    'c_ID': eRASS1['DETUID'].byteswap().newbyteorder(),
    'c_NAME': eRASS1['NAME'].byteswap().newbyteorder(),
    'RA': eRASS1['RA'].byteswap().newbyteorder(),
    'DEC': eRASS1['DEC'].byteswap().newbyteorder(),
    'z': eRASS1['BEST_Z'].byteswap().newbyteorder(),
    'z_type': eRASS1['BEST_Z_TYPE_1'].byteswap().newbyteorder(),
    'cluster_radius_kpc': eRASS1['R500'].byteswap().newbyteorder(),
    'cluster_Velocity_Dispersion': eRASS1['VDISP_BOOT'].byteswap().newbyteorder(),
})
cluster_df_raw['cluster_radius_Mpc'] = cluster_df_raw['cluster_radius_kpc'] / 1000
cluster_df_raw['distance'] = cosmo.comoving_distance(cluster_df_raw['z']).value  # Mpc
cluster_df_raw['cluster_volume'] = get_cluster_volume(cluster_df_raw['cluster_radius_Mpc'])

# Masks
# GALAXIES

# BIG GALAXY
# Apply the masks to the GAMA big DataFrame
gama_full_df = gama_full_raw_df_unmasked[
    (gama_full_raw_df_unmasked['uberclass'] == 1) &  # Classified as galaxy
    (gama_full_raw_df_unmasked['duplicate'] == 0) &  # Unique object
    (gama_full_raw_df_unmasked['mask'] == False) &  # Not masked
    (gama_full_raw_df_unmasked['starmask'] == False) &  # Not star-masked
    (gama_full_raw_df_unmasked['z'] < 0.4) &  # Redshift less than 0.4
    (
            ((gama_full_raw_df_unmasked['RA'] > 129.0) & (gama_full_raw_df_unmasked['RA'] < 141.0) & (
                        gama_full_raw_df_unmasked['DEC'] > -2.0) & (gama_full_raw_df_unmasked['DEC'] < 3.0)) |
            ((gama_full_raw_df_unmasked['RA'] > 174.0) & (gama_full_raw_df_unmasked['RA'] < 186.0) & (
                        gama_full_raw_df_unmasked['DEC'] > -3.0) & (gama_full_raw_df_unmasked['DEC'] < 2.0)) |
            ((gama_full_raw_df_unmasked['RA'] > 211.5) & (gama_full_raw_df_unmasked['RA'] < 223.5) & (
                        gama_full_raw_df_unmasked['DEC'] > -2.0) & (gama_full_raw_df_unmasked['DEC'] < 3.0)) |
            ((gama_full_raw_df_unmasked['RA'] > 339.0) & (gama_full_raw_df_unmasked['RA'] < 351.0) & (
                        gama_full_raw_df_unmasked['DEC'] > -35.0) & (gama_full_raw_df_unmasked['DEC'] < -30.0))
    ) &
    (gama_full_raw_df_unmasked['flux'] >= 5.011928e-05)  # Flux greater than or equal to 5.011928e-05
    ]

# SMALL GALAXY survey masks
gama_df = gama_df_unmasked[
    (gama_df_unmasked['uberclass'] == 1) &  # Classified as galaxy
    (gama_df_unmasked['duplicate'] == 0) &  # Unique object
    (gama_df_unmasked['mask'] == False) &  # Not masked
    (gama_df_unmasked['starmask'] == False) &  # Not star-masked
    (gama_df_unmasked['mstar'] > 0) &  # Stellar mass greater than 0
    (gama_df_unmasked['NQ'] > 2) &  # Reliable redshift
    (gama_df_unmasked['z'] != 0) &  # Redshift not zero
    (gama_df_unmasked['z'] != -9.999) &  # Redshift not -9.999 (invalid value)
    (gama_df_unmasked['z'] < Z_MAX) &  # Redshift less than Z_MAX (define Z_MAX)
    (
            ((gama_df_unmasked['RA'] > 129.0) & (gama_df_unmasked['RA'] < 141.0) & (gama_df_unmasked['DEC'] > -2.0) & (
                        gama_df_unmasked['DEC'] < 3.0)) |
            ((gama_df_unmasked['RA'] > 174.0) & (gama_df_unmasked['RA'] < 186.0) & (gama_df_unmasked['DEC'] > -3.0) & (
                        gama_df_unmasked['DEC'] < 2.0)) |
            ((gama_df_unmasked['RA'] > 211.5) & (gama_df_unmasked['RA'] < 223.5) & (gama_df_unmasked['DEC'] > -2.0) & (
                        gama_df_unmasked['DEC'] < 3.0)) |
            ((gama_df_unmasked['RA'] > 339.0) & (gama_df_unmasked['RA'] < 351.0) & (gama_df_unmasked['DEC'] > -35.0) & (
                        gama_df_unmasked['DEC'] < -30.0))
    ) &
    (gama_df_unmasked['flux'] >= 5.011928e-05)  # Flux greater than or equal to 5.011928e-05
    ]

# CLUSTERS
# Remove rows where 'cluster_Velocity_Dispersion' is NaN
cluster_df_noVelDisp = cluster_df_raw  #.dropna(subset=['cluster_Velocity_Dispersion'])

cluster_df = cluster_df_noVelDisp[
    (cluster_df_noVelDisp['z'] <= 0.4) &
    (
            ((cluster_df_noVelDisp['RA'] > 129.0) & (cluster_df_noVelDisp['RA'] < 141.0) & (
                        cluster_df_noVelDisp['DEC'] > -2.0) & (cluster_df_noVelDisp['DEC'] < 3.0)) |
            ((cluster_df_noVelDisp['RA'] > 174.0) & (cluster_df_noVelDisp['RA'] < 186.0) & (
                        cluster_df_noVelDisp['DEC'] > -3.0) & (cluster_df_noVelDisp['DEC'] < 2.0)) |
            ((cluster_df_noVelDisp['RA'] > 211.5) & (cluster_df_noVelDisp['RA'] < 223.5) & (
                        cluster_df_noVelDisp['DEC'] > -2.0) & (cluster_df_noVelDisp['DEC'] < 3.0)) |
            ((cluster_df_noVelDisp['RA'] > 339.0) & (cluster_df_noVelDisp['RA'] < 351.0) & (
                        cluster_df_noVelDisp['DEC'] > -35.0) & (cluster_df_noVelDisp['DEC'] < -30.0))
    ) &

    (cluster_df_noVelDisp['cluster_radius_kpc'] != -1)
    ]

print(cluster_df_raw.shape)
print(cluster_df.shape)
print(cluster_df_noVelDisp.shape)

# Assuming you have all necessary imports and the constants file loaded correctly
gama_df = calculate_completeness_of_objects(gama_full_df, gama_df)
# LOOOOOOOOOOOOONG
# match the galaxies with the clusters
matched_gama_dataframe = match_galaxies_and_clusters(galaxy_dataframe=gama_df, cluster_dataframe=cluster_df)
matched_gama_dataframe.to_csv('gama_matched_in_erass1.csv')
# load matched galaxies
gama_df = pd.read_csv('gama_matched_in_erass1.csv')
print(gama_df)
counts = gama_df['environment'].value_counts()
ratio = counts.get('ClusterMember', 0) / counts.get('Field', 1) * 100  # Percent
print(f"ClusterMember: {counts.get('ClusterMember', 0)}, Field: {counts.get('Field', 0)}, Ratio: {ratio:.10f}%")

# apply selection function
mass_to_light_histogram_all_galaxies = get_mass_luminosity_histogram(galaxy_df=gama_df)
print(mass_to_light_histogram_all_galaxies)
mass_to_light_histogram_low_mass_galaxies = get_mass_luminosity_histogram(
    galaxy_df=gama_df[gama_df["mstar"] < 10 ** 10])
mass_luminosity_cutoff = get_mass_luminosity_cutoff(gama_df, cut_off_percentage=80)

mass_bin_selection_function = np.logspace(5, 12.25, 1000)
# mass_for_richards_curve = np.logspace(5,12.25, 1000)

selection_function = get_distance_from_mass(mass_bin_selection_function,
                                            log_cutoff_mass_to_light_ratio=mass_luminosity_cutoff)

gama_df['within_selection_function'] = gama_df.apply(
    lambda row: filter_for_selection_function(selection_function, mass_bin_selection_function, row['mstar'],
                                              row['comoving_distance']), axis=1)
# find the general mass function
mass_histogram, mass_histogram_error = get_weighted_mass_histogram(gama_df[gama_df['within_selection_function']],
                                                                   region_name='G09')
stellar_mass_function = mass_histogram / get_region_volume('G09', MASS_BINS[:-1], mass_luminosity_cutoff)
stellar_mass_function_error = mass_histogram_error / get_region_volume('G09', MASS_BINS[:-1], mass_luminosity_cutoff)

# find the cluster mass function

# def get_cluster_volume_sum_from_mass(mass, cluster_df, z_min=0, z_max=999):
#     cut_off_distance = get_distance_from_mass(np.log10(mass))
#     reduced_cluster_dataframe = cluster_df[(cluster_df['distance'] <= cut_off_distance) & (cluster_df['z'] >= z_min) & (cluster_df['z'] <= z_max)]
#     return sum(reduced_cluster_dataframe["cluster_volume"])

# cluster_volumes_by_mass_bin = [get_cluster_volume_sum_from_mass(mass=mass, cluster_dataframe=cluster_df, z_max=0.4) for mass in MASS_BINS]
print(gama_df)


# find the cluster mass function

def get_cluster_volume_sum_from_mass(mass, cluster_df, z_min=0, z_max=0.4, log_cutoff_mass_to_light_ratio=None):
    # If log_cutoff_mass_to_light_ratio is provided, use it; otherwise, use some default behavior
    cut_off_distance = get_distance_from_mass(mass, log_cutoff_mass_to_light_ratio)
    reduced_cluster_dataframe = cluster_df[
        (cluster_df['distance'] <= cut_off_distance) & (cluster_df['z'] >= z_min) & (cluster_df['z'] <= z_max)]
    print(f'for mass {mass} we have {reduced_cluster_dataframe.shape[0]} clusters')
    return sum(reduced_cluster_dataframe["cluster_volume"])


cluster_volumes_by_mass_bin = [
    get_cluster_volume_sum_from_mass(mass=mass, cluster_df=cluster_df, z_max=0.4,
                                     log_cutoff_mass_to_light_ratio=mass_luminosity_cutoff)
    for mass in MASS_BINS
]

# and cluster galaxies
mass_histogram, mass_histogram_error = get_weighted_mass_histogram(
    gama_df[(gama_df["within_selection_function"] == True) & (gama_df["environment"] == "ClusterMember")],
    region_name='G09')
print(mass_histogram)
print(cluster_volumes_by_mass_bin)
stellar_mass_function_cluster = mass_histogram[7:] / cluster_volumes_by_mass_bin[8:]
stellar_mass_function_error_cluster = mass_histogram_error[7:] / cluster_volumes_by_mass_bin[8:]
# plot the cluster function
bin_centers = 0.5 * (MASS_BINS[:-1] + MASS_BINS[1:])
plt.bar(bin_centers[7:] / 0.7 ** 2, stellar_mass_function_cluster, width=np.diff(MASS_BINS[7:]) / 0.7 ** 2,
        edgecolor='black', yerr=stellar_mass_function_error_cluster, capsize=5, alpha=0.2)

plt.plot(bin_centers[19:] / 0.7 ** 2, stellar_mass_function_cluster[12:], color='orange', linewidth=3)
plt.bar(bin_centers[7:] / 0.7 ** 2, stellar_mass_function_cluster, width=np.diff(MASS_BINS[7:]) / 0.7 ** 2,
        edgecolor='black', yerr=stellar_mass_function_error_cluster, capsize=5, alpha=0.0)

plt.xscale('log');
plt.yscale('log')
# plt.xlim(10**7.6, 10**12.5)
# plt.ylim(10**-3, 10**1)
# plt.grid(alpha=0.3)

plt.title("Galaxy Stellar Mass Function for Galaxies in Clusters")
plt.xlabel(r'Galaxy Mass ($M_{\odot}h^{-2}$)')
plt.ylabel(r'Galaxy Abundance ($Mpc^{-3} DEX^{-1} h^{-3}$)')
# plt.savefig("../plots/mass_function_cluster_galaxies.pdf")
# plot the general mass function

bin_centers = 0.5 * (MASS_BINS[:-1] + MASS_BINS[1:])
plt.bar(bin_centers / 0.7, stellar_mass_function * 0.7 ** 3, width=np.diff(MASS_BINS) / 0.7, edgecolor='black',
        capsize=5, alpha=0.3)

plt.plot(bin_centers[10:] / 0.7, stellar_mass_function[10:] * 0.7 ** 3, color='red', linewidth=3,
         label="This Work z < 0.4")
plt.bar(bin_centers / 0.7, stellar_mass_function * 0.7 ** 3, width=np.diff(MASS_BINS) / 0.7, edgecolor='black',
        yerr=stellar_mass_function_error * 0.7 ** 3, capsize=5, alpha=0.0)

plt.xscale('log');
plt.yscale('log')
plt.xlim(10 ** 7.5, 10 ** 12.3)
plt.ylim(10 ** -7, 10 ** -0.5)
plt.grid()

plt.title("Galaxy Stellar Mass Function")
plt.xlabel(r'Galaxy Mass ($M_{\odot}h^{-2}$)')
plt.ylabel(r'Galaxy Abundance ($Mpc^{-3} DEX^{-1} h^{3}$)')
# plt.savefig("../plots/mass_function_alone.pdf")